In [ ]:
import numpy
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import sys
import matplotlib.dates as mdates
import datetime as dt

In [ ]:
def showGraph(data):
    plt.figure()
    plt.plot(data)
    plt.show()
def saveGraph(data, file_name):
    plt.figure(dpi=360)
    plt.plot(data)
    plt.savefig('results/' + file_name)
# convert dataset to x and y
# def create_dataset(dataset, look_back=1):
#     dataX, dataY = [], []
#     for i in range(len(dataset) - look_back - 1):
#         a = dataset[i:(i+look_back), 0]
#         dataX.append(a)
#         dataY.append(dataset[i + look_back, 0])
#     return numpy.array(dataX), numpy.array(dataY)
def relative_error(xs, ys):
    error = 0
    zero = 0
    for i in range(len(xs)):
        if ys[i] != 0:
            error += abs(xs[i]-ys[i])*100 / ys[i]
        else:
            zero += 1
    error /= (len(xs) - zero)
    return error
def read_data(file_name):
    return pandas.read_csv('datas/' + file_name, usecols=[1], sep='|')
def saveGraphWithDate(dates, y_axis, file_name, format_date):
    plt.figure(dpi=360)
    x = [dt.datetime.strptime(d,format_date).date() for d in dates]
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    plt.plot(x,y_axis)
    plt.gcf().autofmt_xdate()
    plt.savefig('results/' + file_name)
def create_dataset(dataset, look_back=1):
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(dataset) - look_back):
        x = dataset[i:(i+look_back), :]
        if i <= int(len(dataset) * 0.67):
            trainX.append(x)
            trainY.append(dataset[i+look_back])
        else:
            testX.append(x)
            testY.append(dataset[i+look_back])
    return numpy.array(trainX), numpy.array(trainY), numpy.array(testX), numpy.array(testY)

In [ ]:
datafile = 'data_stock_market.csv'
result_path = 'raw'
format_date='%d/%m/%Y'
epochs = 10
look_back = 1

print('2. LSTM ')
# dataset = read_data(datafile)
# saveGraph(dataset, result_path + '/data')
dataframe = pandas.read_csv('datas/' + datafile, sep='|')
saveGraphWithDate(dataframe['date'], dataframe['close_price'], result_path + '/data', format_date)
dataset = dataframe['close_price'].values.reshape(dataframe['close_price'].shape[0], 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
# dataset.loc[-1] = [max(dataset.values)*2]
dataset = scaler.fit_transform(dataset)
# dataset = dataset[:-1]

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1

trainX, trainY, testX, testY = create_dataset(dataset, look_back)
trainX = trainX.reshape(trainX.shape[0], look_back, 1)
testX = testX.reshape(testX.shape[0], look_back, 1)
trainY = trainY.reshape(trainY.shape[0], 1)
testY = testY.reshape(testY.shape[0], 1)

In [ ]:
model_input = Input(shape=(look_back, 1))
model_output = LSTM(1, return_sequences=False)(model_input)
model = Model(input=model_input, output=model_output)

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)

# loop predict only the next day and fit to model
print('----- Predict Trend -----')
predict = []
dates_test = dataframe['date'].values[(len(dataset) - len(testY) - 1):(len(dataset)-1)]

for index, today_close_price in enumerate(testX):
    predict_tomorrow = model.predict(numpy.array([today_close_price]))[0]
    predict.append(predict_tomorrow)
    model.fit(numpy.array([today_close_price]), numpy.array([testY[index]]), epochs=epochs, batch_size=1, verbose=0)

In [ ]:
predict = scaler.inverse_transform(predict)
# testX = scaler.inverse_transform(testX[:, 0])
testY = scaler.inverse_transform(testY)

In [ ]:
with open('results/' + result_path + '/result.csv', 'w+') as file:
    file.write('close_price|predict\n')
    for i in range(len(predict)):
        file.write(str(testY[i][0]) + '|' + str(predict[i][0]) + '\n')
plt.figure(dpi=200)
plt.plot(testY, linewidth=0.5)
plt.plot(predict, linewidth=0.5)
plt.show()

In [ ]:
today = scaler.inverse_transform(testX[:, look_back - 1])

In [ ]:
_trend_reallity = [int(numpy.sign(b[0] - a[0])) for a, b in zip(today, testY)]
_trend_predict = [int(numpy.sign(b[0] - a[0])) for a, b in zip(today, predict)]
_correct = [1 if a == b else 0 for a, b in zip(_trend_reallity, _trend_predict)]
print('Percent correct: %.2f' % (100*sum(_correct)/len(_correct)))

In [ ]:
import numpy
import matplotlib
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import sys
import matplotlib.dates as mdates
import datetime as dt

def saveGraph(data, file_name):
    plt.figure(dpi=360)
    plt.plot(data)
    plt.savefig('results/' + file_name)
def relative_error(xs, ys):
    error = 0
    zero = 0
    for i in range(len(xs)):
        if ys[i] != 0:
            error += abs(xs[i]-ys[i])*100 / ys[i]
        else:
            zero += 1
    error /= (len(xs) - zero)
    return error
def read_data(file_name):
    return pandas.read_csv('datas/' + file_name, usecols=[1], sep='|')
def saveGraphWithDate(dates, y_axis, file_name, format_date):
    plt.figure(dpi=360)
    x = [dt.datetime.strptime(d,format_date).date() for d in dates]
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    plt.plot(x,y_axis)
    plt.gcf().autofmt_xdate()
    plt.savefig('results/' + file_name)
def create_dataset(dataset, look_back=1):
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(dataset) - look_back):
        x = dataset[i:(i+look_back), :]
        if i <= int(len(dataset) * 0.67):
            trainX.append(x)
            trainY.append(dataset[i+look_back])
        else:
            testX.append(x)
            testY.append(dataset[i+look_back])
    return numpy.array(trainX), numpy.array(trainY), numpy.array(testX), numpy.array(testY)



datafile = 'data_stock_market.csv'
result_path = 'raw'
format_date='%d/%m/%Y'
epochs = 100

percents_correct = []
look_back = 7
batch_sizes = [1, 2, 3, 4, 5, 6, 8, 10]

for batch_size in batch_sizes:
    dataframe = pandas.read_csv('datas/' + datafile, sep='|')
    dataset = dataframe['close_price'].values.reshape(dataframe['close_price'].shape[0], 1)

    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    trainX, trainY, testX, testY = create_dataset(dataset, look_back)
    trainX = trainX.reshape(trainX.shape[0], look_back, 1)
    testX = testX.reshape(testX.shape[0], look_back, 1)
    trainY = trainY.reshape(trainY.shape[0], 1)
    testY = testY.reshape(testY.shape[0], 1)

    model = Sequential()
    model.add(LSTM(64, input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=2)


    print('----- Predict Trend -----')
    predict = []
    dates_test = dataframe['date'].values[(len(dataset) - len(testY) - 1):(len(dataset)-1)]

    for index, today_close_price in enumerate(testX):
        predict_tomorrow = model.predict(numpy.array([today_close_price]))[0]
        predict.append(predict_tomorrow)
        model.fit(numpy.array([today_close_price]), numpy.array([testY[index]]), epochs=epochs, batch_size=batch_size, verbose=0)

    predict = scaler.inverse_transform(predict)
    testY = scaler.inverse_transform(testY)

    with open('results/' + result_path + '/result.csv', 'w+') as file:
        file.write('close_price|predict\n')
        for i in range(len(predict)):
            file.write(str(testY[i][0]) + '|' + str(predict[i][0]) + '\n')

    today = scaler.inverse_transform(testX[:, look_back - 1])

    _trend_reallity = [int(numpy.sign(b[0] - a[0])) for a, b in zip(today, testY)]
    _trend_predict = [int(numpy.sign(b[0] - a[0])) for a, b in zip(today, predict)]
    _correct = [1 if a == b else 0 for a, b in zip(_trend_reallity, _trend_predict)]
    _percent_correct = 100*sum(_correct)/len(_correct)
    percents_correct.append({'batch_size': batch_size, 'percent': _percent_correct})

    with open('results/' + result_path + '/percents.csv', 'w+') as file:
        file.write('look_back|batch_size|percent\n')
        for i in percents_correct:
            file.write(str(look_back) + '|' + str(i['batch_size']) + '|' + str(i['percent']) + '\n')


Using TensorFlow backend.


Epoch 1/100
7s - loss: 0.0021
Epoch 2/100
7s - loss: 7.6553e-04
Epoch 3/100
7s - loss: 5.8340e-04
Epoch 4/100
6s - loss: 5.9284e-04
Epoch 5/100
6s - loss: 4.8607e-04
Epoch 6/100
6s - loss: 4.2286e-04
Epoch 7/100
6s - loss: 3.2759e-04
Epoch 8/100
7s - loss: 2.8710e-04
Epoch 9/100
6s - loss: 2.9032e-04
Epoch 10/100
5s - loss: 2.7530e-04
Epoch 11/100
5s - loss: 2.7233e-04
Epoch 12/100
5s - loss: 2.8341e-04
Epoch 13/100
6s - loss: 2.6907e-04
Epoch 14/100
6s - loss: 2.7083e-04
Epoch 15/100
5s - loss: 2.6342e-04
Epoch 16/100
5s - loss: 2.5876e-04
Epoch 17/100
5s - loss: 2.4456e-04
Epoch 18/100
5s - loss: 2.6606e-04
Epoch 19/100
5s - loss: 2.4692e-04
Epoch 20/100
5s - loss: 2.6253e-04
Epoch 21/100
5s - loss: 2.4770e-04
Epoch 22/100
5s - loss: 2.4229e-04
Epoch 23/100
5s - loss: 2.5705e-04
Epoch 24/100
5s - loss: 2.4760e-04
Epoch 25/100
5s - loss: 2.4187e-04
Epoch 26/100
5s - loss: 2.4191e-04
Epoch 27/100
5s - loss: 2.5073e-04
Epoch 28/100
5s - loss: 2.4868e-04
Epoch 29/100
5s - loss: 2.3235e-0

1s - loss: 2.4824e-04
Epoch 36/100
1s - loss: 2.2361e-04
Epoch 37/100
1s - loss: 2.3230e-04
Epoch 38/100
1s - loss: 2.1677e-04
Epoch 39/100
1s - loss: 2.2016e-04
Epoch 40/100
1s - loss: 2.1877e-04
Epoch 41/100
1s - loss: 2.3735e-04
Epoch 42/100
1s - loss: 2.4787e-04
Epoch 43/100
1s - loss: 2.2216e-04
Epoch 44/100
1s - loss: 2.4139e-04
Epoch 45/100
1s - loss: 2.3288e-04
Epoch 46/100
1s - loss: 2.3236e-04
Epoch 47/100
1s - loss: 2.2723e-04
Epoch 48/100
1s - loss: 2.3403e-04
Epoch 49/100
1s - loss: 2.2318e-04
Epoch 50/100
1s - loss: 2.2309e-04
Epoch 51/100
1s - loss: 2.1964e-04
Epoch 52/100
1s - loss: 2.2790e-04
Epoch 53/100
1s - loss: 2.3572e-04
Epoch 54/100
1s - loss: 2.3678e-04
Epoch 55/100
1s - loss: 2.2509e-04
Epoch 56/100
1s - loss: 2.2381e-04
Epoch 57/100
1s - loss: 2.3515e-04
Epoch 58/100
1s - loss: 2.2447e-04
Epoch 59/100
1s - loss: 2.4003e-04
Epoch 60/100
1s - loss: 2.1191e-04
Epoch 61/100
1s - loss: 2.2200e-04
Epoch 62/100
1s - loss: 2.0791e-04
Epoch 63/100
1s - loss: 2.2409e-0

1s - loss: 2.1856e-04
Epoch 70/100
1s - loss: 2.1214e-04
Epoch 71/100
1s - loss: 2.1690e-04
Epoch 72/100
1s - loss: 2.3495e-04
Epoch 73/100
1s - loss: 2.1607e-04
Epoch 74/100
1s - loss: 2.1508e-04
Epoch 75/100
1s - loss: 2.2356e-04
Epoch 76/100
1s - loss: 2.1133e-04
Epoch 77/100
1s - loss: 2.2198e-04
Epoch 78/100
1s - loss: 2.1865e-04
Epoch 79/100
1s - loss: 2.1060e-04
Epoch 80/100
1s - loss: 2.1462e-04
Epoch 81/100
1s - loss: 2.0900e-04
Epoch 82/100
1s - loss: 2.2504e-04
Epoch 83/100
1s - loss: 2.1165e-04
Epoch 84/100
1s - loss: 2.0789e-04
Epoch 85/100
1s - loss: 2.3103e-04
Epoch 86/100
1s - loss: 2.0823e-04
Epoch 87/100
1s - loss: 2.0657e-04
Epoch 88/100
1s - loss: 2.1776e-04
Epoch 89/100
1s - loss: 2.0778e-04
Epoch 90/100
1s - loss: 2.1096e-04
Epoch 91/100
1s - loss: 2.2015e-04
Epoch 92/100
1s - loss: 2.0975e-04
Epoch 93/100
1s - loss: 2.2598e-04
Epoch 94/100
1s - loss: 2.0394e-04
Epoch 95/100
1s - loss: 2.1899e-04
Epoch 96/100
1s - loss: 2.0896e-04
Epoch 97/100
1s - loss: 2.1319e-0

0s - loss: 0.0013
Epoch 3/100
0s - loss: 8.3666e-04
Epoch 4/100
0s - loss: 7.8317e-04
Epoch 5/100
0s - loss: 7.5518e-04
Epoch 6/100
0s - loss: 7.5576e-04
Epoch 7/100
0s - loss: 7.7705e-04
Epoch 8/100
0s - loss: 7.2095e-04
Epoch 9/100
0s - loss: 7.1042e-04
Epoch 10/100
0s - loss: 7.1320e-04
Epoch 11/100
0s - loss: 6.6203e-04
Epoch 12/100
0s - loss: 6.8526e-04
Epoch 13/100
0s - loss: 6.6026e-04
Epoch 14/100
0s - loss: 6.9072e-04
Epoch 15/100
0s - loss: 6.6991e-04
Epoch 16/100
0s - loss: 6.4503e-04
Epoch 17/100
0s - loss: 6.1625e-04
Epoch 18/100
0s - loss: 6.2345e-04
Epoch 19/100
0s - loss: 5.8200e-04
Epoch 20/100
0s - loss: 5.4585e-04
Epoch 21/100
0s - loss: 5.2960e-04
Epoch 22/100
0s - loss: 5.5480e-04
Epoch 23/100
0s - loss: 5.2671e-04
Epoch 24/100
0s - loss: 4.9518e-04
Epoch 25/100
0s - loss: 4.6510e-04
Epoch 26/100
0s - loss: 4.8988e-04
Epoch 27/100
0s - loss: 4.7453e-04
Epoch 28/100
0s - loss: 4.4621e-04
Epoch 29/100
0s - loss: 4.3526e-04
Epoch 30/100
0s - loss: 4.4282e-04
Epoch 31/

In [ ]:
percents